In [1]:
import pandas as pd
import re
import json
import csv
import numpy as np
import os.path

In [2]:
# Data Source: https://www.kaggle.com/datasets/albertsuarez/azlyrics?resource=download

with open('../data/kaggle/azlyrics_lyrics_a.csv') as f:
    x = csv.DictReader(f)
    y = pd.DataFrame(x)

In [13]:
for i in list(os.walk('../data/kaggle/'))[0][2]:
    j = i[-5:]
    os.rename('../data/kaggle/'+i,'../data/kaggle/'+j)

In [ ]:
# os.chdir('../data/kaggle/')

for i in list(os.walk('.'))[0][2]:
    with open(i) as fp:
        x = csv.DictReader(fp)
        y = pd.DataFrame(x)
        y[None] = y[None].fillna('').astype(str)
        y['LYRICS'] = y['LYRICS'] + y[None]
        y = y.drop(columns=[None])
        #y.to_parquet

In [34]:
z = pd.DataFrame(columns=['ARTIST_NAME', 'ARTIST_URL', 'SONG_NAME', 'SONG_URL', 'LYRICS'])
for i in list(os.walk('.'))[0][2]:
    if i.endswith('parquet'):
        with open(i) as fp:
            y = pd.read_parquet(i)
            z = pd.concat([z,y],ignore_index=True)


Index(['ARTIST_NAME', 'ARTIST_URL', 'SONG_NAME', 'SONG_URL', 'LYRICS'], dtype='object')

In [39]:
z = z.rename(columns={z.columns[i]:list(map(lambda x : x.lower() , z.columns ))[i] for i in range(len(z.columns)) })

In [47]:
z

,artist_name,artist_url,song_name,song_url,lyrics
0,03 greedo,https://www.azlyrics.com/19/03greedo.html,sweet lady,https://www.azlyrics.com/lyrics/03greedo/sweet...,"its only one, 03, i'm from grape street, where..."
1,03 greedo,https://www.azlyrics.com/19/03greedo.html,mafia business,https://www.azlyrics.com/lyrics/03greedo/mafia...,"you gonna make me put you in a suit and tie, s..."
2,03 greedo,https://www.azlyrics.com/19/03greedo.html,paranoid,https://www.azlyrics.com/lyrics/03greedo/paran...,"we could kill it, yeah, we could, we could, we..."
3,03 greedo,https://www.azlyrics.com/19/03greedo.html,never bend,https://www.azlyrics.com/lyrics/03greedo/never...,"yeah, you lil bitch ass niggas steady speaking..."
4,03 greedo,https://www.azlyrics.com/19/03greedo.html,prayer for my lost,https://www.azlyrics.com/lyrics/03greedo/praye...,"x loaded up bro, never see me in the church, p..."
...,...,...,...,...,...
147867,zug izland,https://www.azlyrics.com/z/zugizland.html,cry,https://www.azlyrics.com/lyrics/zugizland/cry....,"our fire will rise, ringmaster's eyes, funhous..."
147868,zug izland,https://www.azlyrics.com/z/zugizland.html,suicide,https://www.azlyrics.com/lyrics/zugizland/suic...,"hey there killer, where you goin'?, i see you ..."
147869,zug izland,https://www.azlyrics.com/z/zugizland.html,i found you,https://www.azlyrics.com/lyrics/zugizland/ifou...,"life was shitty, can't call it, my karmas, bee..."
147870,zug izland,https://www.azlyrics.com/z/zugizland.html,twilight zone,https://www.azlyrics.com/lyrics/zugizland/twil...,"somewhere in a lonely hotel room, there's a gu..."


In [17]:
x = pd.read_parquet('../data/wikitop100list/artists.parquet')

In [54]:
def initial(artist):
    artist = artist.lower()
    if ' ' in artist:
        art2 = ' '.join('artist'.split()[1:])
    try:
        z[z['artist_name'] == artist].to_parquet(artist+'.parquet')
        
    except:
        try:
            z[z['artist_name'] == art2].to_parquet(artist+'.parquet')
        except:
            print('Failed upload:',artist)
    
        

In [56]:
initial('beatles')

In [57]:
x = pd.read_parquet('the beatles.parquet')
x

,artist_name,artist_url,song_name,song_url,lyrics
index,,,,,


In [83]:
for i in y[None]:
    if type(i) == float:
        print(i)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
